<a href="https://colab.research.google.com/github/chienyuanhung/twitter-sentimometer/blob/master/nlp_sentiment_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re


In [0]:
import pandas as pd
file = '/Users/chien.yuanhung/Documents/GitHub/twitter-sentimometer/resources/trainingandtestdata/training.1600000.processed.noemoticon.csv'

In [5]:
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


In [0]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['training.1600000.processed.noemoticon.csv']), encoding='latin-1', header = None)

In [10]:
df2.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
# take the target and tweet for training, create a new Data Frame

df1 = df2[[0, 5]]
df1.columns = ['target', 'tweet']
df1.head()

,target,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [0]:
# function for getting rid of unwanted characters in tweets

def getCleanTwit(str1):
    words= str1.split()
    clean_words = []
    for i in words:
        if re.findall(r'^http', i):
            continue
        else:
            clean_words.append(i)
    text = u' '.join(x for x in clean_words)
    text = re.sub("http*:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]", u"", text)
    text = re.sub("\@[\_0-9a-zA-Z]+\:?", u"", text)
    text = re.sub("\&amp\;?", u"", text)
    text = re.sub("[\:\.]{1,}$", u"", text)
    text = re.sub("^RT\:?", u"", text)

    return text


In [0]:
tweets = df1.tweet.to_list()

In [0]:
# the list for clean tweet
all_tweets = []
for i in tweets:
    twi = getCleanTwit(i)
    all_tweets.append(twi)

In [0]:
# import the library to tokenize and remove stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stopWords = set(stopwords.words('english'))

In [0]:
# remove the stopwords from twees
stopwords_removed = []

for i in all_tweets:
    text = []
    for j in i.split():
        j = j.lower()
        j = j.replace('.', '')
        j = j.replace(',', '')
        j = j.replace('?', '')
        if j not in stopWords:
            text.append(j)
            
    stopwords_removed.append(u' '.join(x for x in text))
        



In [21]:
# put the clean tweets but to the dataframe
df1['clean_tweet'] = stopwords_removed
df1.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,target,tweet,clean_tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",- awww that's bummer shoulda got david carr th...
1,0,is upset that he can't update his Facebook by ...,upset can't update facebook texting might cry ...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50% rest go...
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving i'm mad can't see


In [0]:
label = df1['target'].to_list()
labels = label[0: 200000] + label[800000:1000000]

features = stopwords_removed[0: 200000] + stopwords_removed[800000:1000000]



In [0]:
# use TF_IDF vectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
features= vectorizer.fit_transform(features).toarray()

In [1]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)


NameError: ignored

In [56]:
# train the data with random forest classifier
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators =200, min_samples_split= 10, min_samples_leaf =4, max_features = 'auto', max_depth = 70, bootstrap= True)
text_classifier.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=70, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
predictions = text_classifier.predict(X_test)


In [58]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[12161  7752]
 [ 3524 16563]]
              precision    recall  f1-score   support

           0       0.78      0.61      0.68     19913
           4       0.68      0.82      0.75     20087

    accuracy                           0.72     40000
   macro avg       0.73      0.72      0.71     40000
weighted avg       0.73      0.72      0.71     40000

0.7181
